In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.linalg import eigvals
from itertools import combinations

In [2]:
def generate_subsets(n):
    # Generate all possible subsets of size 1 to n.
    for size in range(1, n + 1):
        for subset in combinations(range(n), size):
            yield subset
            # print(subset)

def check_p(matrix):
    n = len(matrix)

    # Compute the determinant of the whole matrix first.
    determinant = np.linalg.det(matrix)
    if determinant <= 0:
        return "Not a p-matrix"  # Return immediately if the whole matrix has a non-positive determinant

    # Check the determinants of diagonal elements (smallest subset) next.
    for i in range(n):
        submatrix = matrix[i, i]
        if submatrix <= 0:
            return "Not a p-matrix"  # Return immediately if any diagonal element is non-positive

    # Initialize the generator to get subsets.
    subset_generator = generate_subsets(n)

    # Check the determinants of other subsets.
    for subset in subset_generator:
        if len(subset) > 1:
            submatrix = matrix[np.ix_(subset, subset)]
            determinant = np.linalg.det(submatrix)
            if determinant <= 0:
                return "Not a p-matrix"  # Return immediately if a non-positive determinant is found

    return "All principal minors are positive -> p-matrix"


In [3]:
import numpy as np

def generate_non_symmetric_matrix(n):
    # Generate a random matrix with values between -1 and 1
    matrix = np.random.uniform(-1, 1, (n, n))

    # Set diagonal elements to 1
    np.fill_diagonal(matrix, 1)

    # Check sub-matrix determinants and adjust as needed
    for i in range(n):
        for j in range(i + 1, n):
            sub_matrix = matrix[:j+1, :j+1]  # Extract sub-matrix
            while np.linalg.det(sub_matrix) < 0:
                # Find a random element in the sub-matrix
                row_idx, col_idx = np.random.randint(0, j+1), np.random.randint(0, j+1)
                # Flip the sign of the element
                matrix[row_idx, col_idx] *= -1
    
    return matrix

# Define the size of the matrix (change n to your desired value)
n = 4
result_matrix = generate_non_symmetric_matrix(n)
print(result_matrix)

[[ 1.         -0.47089793 -0.94307132 -0.65566088]
 [-0.4496555   1.          0.6501008   0.43143654]
 [-0.24755936  0.95556381  1.          0.57089045]
 [-0.89512668  0.17840719  0.58751799  1.        ]]


In [4]:
def schur_complement(matrix, block_indices):
    A = matrix[block_indices[0]:block_indices[1], block_indices[0]:block_indices[1]]
    B = matrix[block_indices[0]:block_indices[1], block_indices[1]:]
    C = matrix[block_indices[1]:, block_indices[0]:block_indices[1]]
    D = matrix[block_indices[1]:, block_indices[1]:]

    inv_D = np.linalg.inv(D)
    S = A - B @ inv_D @ C

    return S

# Create a random 4x4 matrix
matrix = generate_non_symmetric_matrix(4)

# Define the block indices for partitioning (e.g., block at row 2 and column 2)
block_indices = (2, 4)

# Compute the Schur complement
schur_matrix = schur_complement(matrix, block_indices)

print("Original Matrix:")
print(matrix)

print("\nSchur Complement:")
print(schur_matrix)

Original Matrix:
[[ 1.          0.41180866 -0.95501689  0.33562446]
 [-0.72924636  1.          0.04034673 -0.12964737]
 [ 0.14018943  0.19382237  1.         -0.62063751]
 [-0.50179487  0.48002998 -0.30126106  1.        ]]

Schur Complement:
[[ 1.         -0.62063751]
 [-0.30126106  1.        ]]


In [5]:
check_p(result_matrix)

'All principal minors are positive -> p-matrix'

In [10]:
n = 50
while check_p(result_matrix) == "Not a p-matrix":
    result_matrix = generate_non_symmetric_matrix(n)
    check_p(result_matrix)

KeyboardInterrupt: 